In [1]:
import sys
sys.path.insert(0, "/home/xcaruso/travail/articles/gabidulin/sage")

In [2]:
from Ore_polynomial_ring_constructor import OrePolynomialRing1
from Ore_polynomial_element import OrePolynomial1

In [3]:
def llcm_multiple(Ore, L):
    n = len(L)
    if n == 1:
        return L[0]
    elif n == 2:
        return L[0].llcm(L[1])
    else:
        temp = L[0].llcm(L[1])
        for i in range(2, len(L)):
            temp = temp.llcm(L[i])
        return temp

In [22]:
A.<t> = GF(3)[]
K = A.fraction_field()
theta = End(K).identity()
delta = K.derivation()
D = OrePolynomialRing1(K, theta, delta, "X")

In [23]:
def gamma(c, g, P):
    res = []
    for i in range(len(c)):
        res += [P(c[i], g[i])]
    return vector(res)
            

In [24]:
def erreur(A, n, rang):
    p = A.characteristic()
    t = A.gen()
    f1 = [A.random_element() for _ in range(rang/2)]
    f2 = [A.random_element() for _ in range(rang/2)]
    err = [sum(A.random_element()(t^p) * f1[j] for j in range(rang/2)) for _ in range(n)] + [sum(A.random_element()(t^p) * f1[j] for j in range(rang/2)) for _ in range(n)]
    return vector(err)

In [25]:
def Ann(Ore, delta, theta, c, L):
    n = len(L)
    L1 = []
    for i in range(n):
        if L[i] == 0:
            L1 += [OrePolynomial1(Ore, [0, 1])]
        else:
            temp = (delta(L[i]) + c[i]*theta(L[i]))
            L1 += [OrePolynomial1(Ore, [-temp/L[i], 1])]
    return llcm_multiple(Ore, L1)

In [26]:
def Inter(Ore, delta, theta, c, g, y):
    res = OrePolynomial1(Ore, [0])
    for i in range(len(g)):
        g1 = g[:i] + g[i+1:]
        c1 = c[:i] + c[i+1:]
        temp = Ann(Ore, delta, theta, c1, g1)
        if temp(c[i], g[i]) != 0:
            res += OrePolynomial1(Ore, [y[i]/temp(c[i], g[i])]) * temp 
    return res

In [27]:
def decodage(Ore, G0, G1, n, k):
    R = [G0, G1]
    U = [OrePolynomial1(Ore, [1]), OrePolynomial1(Ore, [0])]
    V = [OrePolynomial1(Ore, [0]), OrePolynomial1(Ore, [1])]
    i = 1
    while R[i].degree() >= (n+k)/2:
        Q, r = R[i-1].div_euc(R[i])
        U += [U[i-1] - Q*U[i]]
        V += [V[i-1] - Q*V[i]]
        R += [r]
        i += 1
    Q, r = R[i].div_euc(V[i], sens="left")
    if r.is_zero() and Q.degree() < k:
        return Q
    else:
        raise ValueError("Erreur de décodage, trop d'erreurs.")

In [48]:
c = [i for i in range(2) for _ in range(3)]
g = [t^j for i in range(2) for j in range(3)]

In [49]:
c

[0, 0, 0, 1, 1, 1]

In [50]:
g

[1, t, t^2, 1, t, t^2]

In [51]:
P = OrePolynomial1(D, [1,t^2])

In [52]:
P

t^2*X + 1

In [47]:
P.__call__??

In [53]:
m = gamma(c, g, P)
m

(1, t^2 + t, 2*t^3 + t^2, t^2 + 1, t^3 + t^2 + t, t^4 + 2*t^3 + t^2)

In [65]:
err = vector([1,t^3,2*t^3,t+1,0,t^4+t^3])

In [66]:
C = m + err
C

(2, t^3 + t^2 + t, t^3 + t^2, t^2 + t + 2, t^3 + t^2 + t, 2*t^4 + t^2)

In [67]:
%time L = Ann(D, delta, theta, c, g)
L

CPU times: user 40 ms, sys: 4 ms, total: 44 ms
Wall time: 43.4 ms


X^6 + 2*X^3

In [82]:
%time Ptilde = Inter(D, delta, theta, c, g, C)
Ptilde

CPU times: user 148 ms, sys: 12 ms, total: 160 ms
Wall time: 140 ms


((2*t^4 + t^3 + 2*t^2)/(t + 2))*X^5 + (2*t^4 + t^2)*X^4 + (2*t^4 + t^3 + 2*t)*X^3 + ((2*t^5 + 2*t^2)/(t + 2))*X^2 + (t^3 + t^2 + 2*t)*X + 2

In [92]:
Ptilde = OrePolynomial1(D, [2, t^3 + t^2 + 2*t, 2*t^4 + t^3 + 2*t^2, 2*t^4 + t^3 + 2*t, 2*t^4+t^2])
Ptilde

(2*t^4 + t^2)*X^4 + (2*t^4 + t^3 + 2*t)*X^3 + (2*t^4 + t^3 + 2*t^2)*X^2 + (t^3 + t^2 + 2*t)*X + 2

In [93]:
Ptilde(1,t^2)

2*t^4 + t^2

In [94]:
%time Q = decodage(D, L, Ptilde, len(c), 2)

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 11.4 ms


In [80]:
P

t^2*X + 1

In [81]:
Q

t^2*X + 1

In [95]:
Ptilde

(2*t^4 + t^2)*X^4 + (2*t^4 + t^3 + 2*t)*X^3 + (2*t^4 + t^3 + 2*t^2)*X^2 + (t^3 + t^2 + 2*t)*X + 2

In [96]:
L

X^6 + 2*X^3

In [98]:
Q, R = L.div_euc(Ptilde)

In [99]:
Q

(2/(t^4 + 2*t^2))*X^2 + (1/(t^4 + 2*t^3 + 2*t^2 + t))*X

In [100]:
R

(1/(t^2 + 2))*X^3 + ((2*t^3 + t^2 + 2)/(t^5 + 2*t^4 + 2*t^3 + t^2))*X^2 + (1/(t^3 + t^2))*X

In [108]:
R = (t^5 + 2*t^4 + 2*t^3 + t^2)*Q*Ptilde - (t^5 + 2*t^4 + 2*t^3 + t^2)*L

In [109]:
U = (t^5 + 2*t^4 + 2*t^3 + t^2)*Q

In [110]:
V = t^5 + 2*t^4 + 2*t^3 + t^2

In [111]:
U*Ptilde - V*L

(2*t^3 + t^2)*X^3 + (t^3 + 2*t^2 + 1)*X^2 + (2*t^2 + 2*t + 2)*X

In [112]:
R

(2*t^3 + t^2)*X^3 + (t^3 + 2*t^2 + 1)*X^2 + (2*t^2 + 2*t + 2)*X

In [114]:
U*P

(2*t^3 + t^2)*X^3 + (t^3 + 2*t^2 + 1)*X^2 + (2*t^2 + 2*t + 2)*X